In [1]:
import torch

from src import read_dicom, Detector, Siddon

In [2]:
print(torch.__version__)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

1.12.0


In [3]:
volume, spacing = read_dicom("../data/cxr/")
isocenter = [0.0, 0.0, 0.0]

source = [-10., -10., -15.]
center = [400., 375., 350.]

detector = Detector(
    source=source,
    center=center,
    height=101,
    width=101,
    delx=0.1,
    dely=0.1,
    device=device
)

source = detector.source
rays = detector.make_xrays()

siddon = Siddon(spacing, isocenter, volume, device)
%timeit siddon.raytrace(source, rays)

749 ms ± 3.67 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
